In [ ]:
!pip install sastrawi
!pip install swifter

In [ ]:
import pandas as pd
import tweepy as tw
import math
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
api_key = 'FSSjIX9PWtgPkmxAZr5oNN70v'
api_key_secret = 'ySTNsffGwh8IlALmnhyzQN0gCsD4RF3gW75rQZDNtk2402K5FK'
access_token = '1314360748865249280-xTyYCvheTzvcvo6RNYD7XUSorTRvB3'
access_token_secret = 'lSvETMGce9Q9hNr5QUEOyeZ5YBY3ohE5eoyQcoMn5VHgu'

auth = tw.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

def get_tweet(query, total):

  data_tweets = []
  places = api.geo_search(query="Indonesia", granularity="country")
  place_id = places[0].id
  
  filter_query = query + ' place:%s' % place_id +' -filter:retweets'
  

  while(total > 0):
    iteration = math.ceil(total / 100)

    for x in range(0, iteration):
      if x == 0:
        tweets = api.search(
            q=filter_query,
            lang='id',
            count=(100 if total >= 100 else total),
            tweet_mode='extended'
        )
      else:
        tweets = api.search(
            q=filter_query,
            lang='id',
            count=(100 if total >= 100 else total),
            max_id=str(max_id - 1),
            tweet_mode='extended'
        )
      
      data_tweets += [[tweet.id, tweet.user.name, tweet.user.location, tweet.full_text, tweet.created_at] for tweet in tweets]
      max_id = tweets[-1].id

      total = total - len(tweets)

  return data_tweets

In [ ]:
def remove_punct(text):
    text = text.lower()
    text = re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE)  # link
    text = re.sub(r'@[A-Za-z0-9_]+', ' ', text)  # menghilangkan @
    text = re.sub(r'[^a-zA-Z0-9_]', ' ', str(text))  # huruf/angka/underline
    text = re.sub(r'\s\s+', ' ', text)  # whitespace
    text = re.sub(r'\b\w(1,2)\b', ' ', text)  # batas antara kata dan non-kata
    text = re.sub(r'[0-9]+', ' ', text)  # angka
    return text

def tokenize(text):
  
  return word_tokenize(text)

def stopword_removal(text):
  indonesian_stopwords = stopwords.words('indonesian')
  indonesian_stopwords.extend(
      [
       "yg", "dg", "rt", "dgn", "ny", "d", 'klo',
       'kalo', 'amp', 'biar', 'bikin', 'bilang',
       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
       '&amp', 'yah', 'jln', 'bs'
      ]
  )

  indonesian_stopwords = set(indonesian_stopwords)

  text = [w for w in text if not w in indonesian_stopwords]
  
  return text

def stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = [stemmer.stem(token) for token in text]
    return text

def preprocessing(dataset):
  dataset['prepro'] = dataset['tweet'].apply(remove_punct)
  dataset['prepro'] = dataset['prepro'].apply(tokenize)
  dataset['prepro'] = dataset['prepro'].apply(stopword_removal)
  
  #Remove Duplicate
  dataset = dataset[dataset.applymap(lambda x: x[0] if isinstance(x, list) else x).duplicated('prepro')]
  
  dataset['prepro'] = dataset['prepro'].swifter.apply(stemming)
  
  return dataset

In [ ]:
total_tweet = 2000
keyword = 'covid'

# Crawling Data
tweets = get_tweet(keyword, total_tweet)

df = pd.DataFrame(data=tweets, columns=['id', 'user', 'location', 'tweet', 'created_at'])
df

# Dataset Preparation
dataset = df[['id','user','tweet','created_at']]

# Preprocessing
cleaned_data = preprocessing(dataset)

# Export Data
cleaned_data.to_csv('Prepro1.csv')